# Startup for Austria
This code is to be included at the beginning of `calib_AUS` and `simu_AUS`.

In [1]:
t0 = time() ## start chrono
dir_module = joinpath(pwd(),"src")
dir_graph = joinpath(pwd(),"../../graphs")
    
########### main module
using NBInclude
@nbinclude(joinpath(dir_module,"AgeDisc.ipynb"))
using Main.AgeDisc

########### other packages
using QuadGK, DataFrames, PyPlot

## _PRECOMPILE_ is whether to run a few functions so that they are already compiled
if !@isdefined(_PRECOMPILE_)
    _PRECOMPILE_ = true
end

true

# Moments

In [2]:
data_aus = DataFrame([
(46.125, 12.4),
(46.375, 13.8),
(46.625, 13.9),
(46.875, 15.7),
(47.125, 11.4),
(47.375, 17.4),
(47.625, 11.),
(47.875, 17.2),
(48.125, 17.3),
(48.375, 14.1),
(48.625, 18.7),
(48.875, 12.),
(49.125, 12.),
(49.375, 16.1),
(49.625, 13.4),
(49.875, 15.2),
(50.125, 29.4),
(50.375, 26.1),
(50.625, 28.9),
(50.875, 25.7),
(51.125, 32.1),
(51.375, 32.8),
(51.625, 36.2),
(51.875, 30.),
(52.125, 29.9),
(52.375, 30.7),
(52.625, 21.9),
(52.875, 23.8),
(53.125, 26.6),
(53.375, 31.6),
(53.625, 24.8),
(53.875, 31.)], [:age, :rate])

data_aus.rate *= 12/52 ## convert to months    
data_aus.age *=12


moments_target, Age_vec = let
    Age_bds = (48, 52) .*12 ## window to match the moments
   # myselec_step1 = [i % 3 == 0 for i in 1:length(mydata.age)]
    myselec = (Age_bds[1] .<= data_aus.age .<= Age_bds[2]) # .& myselec_step1 
    myselec = Age_bds[1] .<= data_aus.age .<= Age_bds[2]
    (data_aus.rate[myselec], data_aus.age[myselec])
end

function moments_austria(V::Surplus,C::Country2;
             a_range=Age_vec)
    ## ex: for age a in data, we compute the average between D(a-1.5) and D(a+1.5)
    Y_vec = zeros(1+length(a_range))
    toadd = expectedD(a_range[1]-1.5,V,C)
    Y_vec[1] += toadd/2
    for (i,a) in enumerate(a_range)
        if a+1.5==C.A
            toadd1 = expectedD(a+1.499,V,C)
            Y_vec[i] += toadd1/2
            toadd2 = expectedD(a+1.5,V,C)
            Y_vec[i+1] += toadd2/2
        else
            toadd = expectedD(a+1.5,V,C)
            Y_vec[i] += toadd/2
            Y_vec[i+1] += toadd/2
        end
    end
    Y_vec[1:length(a_range)]
end

#=
function moments_austria(V::Surplus,C::Country2)
    Y_vec = [expectedD(a,V,C) for a in Age_vec]
    return Y_vec
end
=#

moments_austria (generic function with 1 method)

# Parameters

In [3]:
C0 = let
    Country2(
        A = 50. * 12, Amin = 46. *12, Amax = 54. *12, Aret=65. * 12, 
        r = 0.0048, # 5 % annual interest rate, time unit = month
        α = NaN, m = NaN, q0=0.02, ## 1-exp(-12q) should be close to 0.22~0.25
        qa =NaN, χ=0.59, ##59% of workers receive UI benefits
        λ0 = 1/(39*12/52), ## 39 weeks eligibility
        λ1 = 1/(209*12/52), ## 209 weeks

        bUI = 0.6,   # replacement rate for median wage earner, Lalive
        b0 = 0.7 * 0.6, # median UA for the median UI, Lalive
        B1=NaN, B2=NaN, B1dif=NaN
    )    
end


function update_Country2(;α=C0.α, m=C0.m, q0=C0.q0, qa=C0.qa, χ=C0.χ, Aret=C0.Aret)
    (B1, B1dif, B2) = Baux(α=α, m=m) 
    Country2(A=C0.A, Amin=C0.Amin, Amax=C0.Amax, Aret=Aret,
                    r=C0.r, q0=q0, qa=qa, m=m, λ1=C0.λ1, λ0=C0.λ0, α=α,χ=χ,
                    bUI=C0.bUI, b0=C0.b0,
                    B1=B1, B1dif=B1dif, B2=B2)
end

if _PRECOMPILE_
    let 
        austria = update_Country2(α=0.0185, m=0.0681, qa=0.) 
        a=49.75*12
        V = find_Surplus(austria,reltol=1e-3, findΨ=false) 
        expectedB(a, V, austria)
        expectedD(a, V, austria)
    end  
end

3.6114668219198656

In [4]:
println("C0 is $C0")
t1 = time()
nsec = (t1-t0)
println("Startup run in $nsec secs.")

C0 is Country2(600.0, 552.0, 648.0, 780.0, 0.0048, NaN, 0.02, NaN, NaN, 0.02073365231259968, 0.1111111111111111, 0.59, 0.6, 0.42, NaN, NaN, NaN)
Startup run in 82.62006378173828 secs.
